In [1]:
import json

# Wikidata

In [2]:
with open('wikidata/wizards/data/info.json') as f:
    info = json.load(f)

In [3]:
with open('wikidata/wizards/data/relation.json') as f:
    relation = json.load(f)

In [4]:
info[0]

{'character': 'http://www.wikidata.org/entity/Q173998',
 'characterLabel': 'Ron Weasley',
 'genderLabel': 'male',
 'blood_statusLabel': 'pureblood',
 'DOBLabel': '1980-03-01T00:00:00Z',
 'POBLabel': 'Devon',
 'hair_colorLabel': 'red hair',
 'educationLabel': 'Hogwarts',
 'employerLabel': "Weasleys' Wizard Wheezes",
 'membershipLabel': "Dumbledore's Army"}

## merge entity wikidata

- needeed merged first since there the way wikidata display one entity with different attributes values
- below is an example with entity `Ron Weasley` with `red hair` and `orange hair`

In [5]:
info[0]['characterLabel'],info[0]['hair_colorLabel'],info[1]['characterLabel'],info[1]['hair_colorLabel']

('Ron Weasley', 'red hair', 'Ron Weasley', 'orange hair')

### attritbutes

In [6]:
info[0].keys()

dict_keys(['character', 'characterLabel', 'genderLabel', 'blood_statusLabel', 'DOBLabel', 'POBLabel', 'hair_colorLabel', 'educationLabel', 'employerLabel', 'membershipLabel'])

In [7]:
wikidata = {}

indicator = -1
for i,entity0 in enumerate(info):
    if i < indicator:
        continue
        
    name = entity0['characterLabel']
    wikidata[name] = {}
    
    idx = [i]
    for j,entity1 in enumerate(info[i+1:]):
        if entity1['characterLabel'] == name:
            idx.append(j+i+1)
        else:
            indicator = j+i+1
            break
            
    wikidata[name]['info'] = idx

In [8]:
relation[0].keys()

dict_keys(['character', 'characterLabel', 'fatherLabel', 'motherLabel', 'spouseLabel', 'childLabel', 'siblingLabel', 'in_relationshipLabel', 'enemyLabel'])

In [9]:
indicator = -1
for i,entity0 in enumerate(relation):
    if i < indicator:
        continue
        
    name = entity0['characterLabel']
    
    idx = [i]
    for j,entity1 in enumerate(relation[i+1:]):
        if entity1['characterLabel'] == name:
            idx.append(j+i+1)
        else:
            indicator = j+i+1
            break
            
    wikidata[name]['relation'] = idx

# Fandom data

In [10]:
relation[505]

{'character': 'http://www.wikidata.org/entity/Q106392724',
 'characterLabel': 'Eldred Worple'}

In [11]:
with open('fandom/fandom_data/Gryffindors.json') as f:
    Gryffindors = json.load(f)
with open('fandom/fandom_data/Hufflepuffs.json') as f:
    Hufflepuffs = json.load(f)
with open('fandom/fandom_data/Ravenclaws.json') as f:
    Ravenclaws = json.load(f)
with open('fandom/fandom_data/Slytherins.json') as f:
    Slytherins = json.load(f)

In [12]:
max_idx = 0
max_len = 0

for i in range(len(Gryffindors)):
    l = len(Gryffindors[i].keys())
    
    if l > max_len:
        max_idx = i
        max_len = l

In [13]:
Gryffindors[max_idx].keys()

dict_keys(['name', 'Biographical information', 'Physical information', 'Relationship information', 'Magical characteristics', 'Affiliation'])

In [14]:
for k in list(Gryffindors[max_idx].keys())[1:]:
    print(list(Gryffindors[max_idx][k].keys()))

['Born', 'Blood status', 'Nationality', 'Also known as', 'Title(s)']
['Species', 'Gender', 'Hair colour', 'Skin colour']
['Family members']
['Boggart', 'Wand', 'Patronus']
['House', 'Loyalty']


# Export Data

## entity attributes

| index | name  | blood_status |hair_color | eye_color | wand | patronus | occupation | loyalty | relation+| relation- | house |
| :- | :-   | :-          | :-       | :-       | :-  | :-      | :-  | :-  | :- | :- | -: |
| ***Int*** | ***str*** |  ***str***| ***str***  |  ***str*** |  ***List***  |  ***str*** | ***List***  |    ***List***   |   ***List***  | ***List*** | ***str*** | 






In [15]:
def merge_fandom(data,house):
    
    results_ = []
    
    for i in data:

        name = i['name']
        # name = i['name'].split(' ')[0] + ' ' + i['name'].split(' ')[-1]
        entity = { 'name'     : name,
                    'blood_status': '',
                    'hair_color'  : '',
                    'eye_color'   : '',
                    'patronus'    : '',
                    'wand'        : [],
                    'occupation'  : [],
                    'loyalty'     : [],
                    'relation+'   : [],
                    'relation-'   : []
                            }


        if 'Biographical information' in i.keys():      
            if 'Blood status' in i['Biographical information'].keys():
                entity['blood_status'] = i['Biographical information']['Blood status']

        if 'Physical information' in i.keys():    
            if 'Hair colour' in i['Physical information'].keys():
                entity['hair_color'] = i['Physical information']['Hair colour']
            if 'Eye colour' in i['Physical information'].keys():
                entity['eye_color'] = i['Physical information']['Eye colour']

        if 'Magical characteristics' in i.keys():
            if 'Patronus' in i['Magical characteristics'].keys():
                entity['patronus'] = i['Magical characteristics']['Patronus']
            if 'Wand' in i['Magical characteristics'].keys():
                entity['wand'] = i['Magical characteristics']['Wand']

        if 'Affiliation' in i.keys():    
            if 'Occupation' in i['Affiliation'].keys():
                entity['occupation'] = i['Affiliation']['Occupation']
            if 'Loyalty' in i['Affiliation'].keys():
                entity['loyalty'] = i['Affiliation']['Loyalty']

        if 'Relationship information' in i.keys():
            if 'Family members' in i['Relationship information'].keys():
                entity['relation+'] = i['Relationship information']['Family members']
                # entity['relation'] = [x.split(' ()')[0] for x in i['Relationship information']['Family members']]

        entity['house'] = house
        
        #去除独立的node
        #if entity['relation'] != []:
        results_.append(entity)
        
    return results_

In [16]:
data_ = []
data_ += merge_fandom(Gryffindors,'Gryffindors')
data_ += merge_fandom(Hufflepuffs,'Hufflepuffs')
data_ += merge_fandom(Ravenclaws,'Ravenclaws')
data_ += merge_fandom(Slytherins,'Slytherins')

In [17]:
#如果去除relation为空的 是204
len(data_)

812

In [18]:
import requests
from time import sleep

In [19]:
for i in range(len(data_)):
    person = data_[i]
    name = person['name']
    wikidata_key = person['name']
    name_lst = name.split(' ')
    
    if len(name_lst) == 3:
        name = name_lst[0] + '_' + name_lst[-1]
        wikidata_key = name_lst[0] + ' ' + name_lst[-1]
    else:
        name = name.replace(' ', '_')
        
    if wikidata_key == 'Ronald Weasley': #暂时只看见了这一个 wikidata里名字是简写
        wikidata_key = 'Ron Weasley'
    if wikidata_key in wikidata.keys():
        relations = wikidata[wikidata_key]['relation']
        
        for r in relations:
            try:
                enemy = relation[r]['enemyLabel']
                if enemy not in data_[i]['relation-']:
                    data_[i]['relation-'].append(enemy)
            except KeyError:
                pass
        continue

    sparql_query = """
        prefix schema: <http://schema.org/>
        SELECT ?itemLabel ?enemyLabel
        WHERE {
            <https://en.wikipedia.org/wiki/""" + name + """> schema:about ?item .
            
            ?item wdt:P7047 ?enemy .
            SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
        }
    """
    
    url = 'https://query.wikidata.org/sparql'
    
    sleep(1)
    r = requests.get(url, params={'format': 'json', 'query': sparql_query})
    try:
        curr_data = r.json()
    except:
        print(r)
        continue
        
    for r in curr_data['results']['bindings']:
        enemy = r['enemyLabel']['value']
        if enemy not in data_[i]['relation-']:
            data_[i]['relation-'].append(enemy)        

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>


In [20]:
count = 0
for p in data_:
    if len(p['relation-']) > 0:
        count += 1
count

10

In [21]:
with open('death_eaters/death_eaters_data/death_eaters.json') as f:
    death_eaters_raw = json.load(f)

In [48]:
death_eaters_data_ = merge_fandom(death_eaters_raw,'Death_Eaters')

In [49]:
death_eaters_data_

[{'name': 'Alecto Carrow',
  'blood_status': 'Pure-blood or Half-blood',
  'hair_color': '',
  'eye_color': '',
  'patronus': 'None',
  'wand': 'Unknown length, wood and core materials',
  'occupation': ['Professor of Muggle Studies at Hogwarts (1997—1998)',
   'Deputy Headmistress at Hogwarts (1997—1998)'],
  'loyalty': ['Death EatersLord Voldemort',
   'Lord Voldemort',
   'Carrow family',
   'Hogwarts School of Witchcraft and WizardrySlytherin',
   'Slytherin'],
  'relation+': ['Amycus Carrow (brother)',
   'Flora Carrow (likely)',
   'Hestia Carrow (likely)'],
  'relation-': [],
  'house': 'Death_Eaters'},
 {'name': 'Rodolphus Lestrange',
  'blood_status': 'Pure-blood',
  'hair_color': 'Dark',
  'eye_color': 'Dark',
  'patronus': 'None',
  'wand': [],
  'occupation': [],
  'loyalty': ['Lord VoldemortDeath Eaters',
   'Death Eaters',
   'Lestrange family',
   'House of Black',
   'Hogwarts School of Witchcraft and Wizardry (formerly)Slytherin',
   'Slytherin'],
  'relation+': ['Rado

In [50]:
# Convert Tom Marvolo Riddle to more common name Lord Voldemort
for i in range(len(death_eaters_data_)):
    if death_eaters_data_[i]['name'] == 'Tom Marvolo Riddle':
        death_eaters_data_[i]['name'] = 'Lord Voldemort'

In [51]:
for i in range(len(data_)):
    p = data_[i]
    loyalties = p['loyalty']
    p_name = p['name']
    
    if type(loyalties) == str:
        if 'Order of the Phoenix' in loyalties or "Dumbledore's Army" in loyalties:
            for j in range(len(death_eaters_data_)):
                e = death_eaters_data_[j]
                e_name = e['name']
                
                if e_name not in data_[i]['relation-']:
                    data_[i]['relation-'].append(e_name)
    else:
        for loyalty in loyalties:
            if 'Order of the Phoenix' in loyalty or "Dumbledore's Army" in loyalty:
                for j in range(len(death_eaters_data_)):
                    e = death_eaters_data_[j]
                    e_name = e['name']
                
                    if e_name not in data_[i]['relation-']:
                        data_[i]['relation-'].append(e_name)
                break

In [52]:
has_relation_data_ = []

for p in data_:
    relations_count = len(p['relation+']) + len(p['relation-'])
    
    if relations_count > 0:
        has_relation_data_.append(p)

In [53]:
len(has_relation_data_)

240

In [54]:
has_relation_data = dict(enumerate(has_relation_data_))

In [55]:
death_eaters_data = {}
key = len(has_relation_data)
for i in death_eaters_data_:
    death_eaters_data[key] = i
    key += 1

In [56]:
import csv

In [58]:
with open('nodes.csv', 'w', newline='') as f:
    f_writer = csv.writer(f)
    f_writer.writerow(['id','name', 'blood_status', 'hair_color', 'eye_color', 'patronus', 'wand', 'occupation', 'loyalty', 'house'])
    
    idx = 0
    for i in range(len(has_relation_data)):
        idx = i
        row = has_relation_data[i]
        res = [
            str(i), 
            row['name'], 
            row['blood_status'], 
            row['hair_color'], 
            row['eye_color'], 
            row['patronus'], 
            row['wand'],
            row['occupation'],
            row['loyalty'],
            row['house']
        ]
        for j in range(len(res)):
            if len(res[j]) == 0:
                res[j] = 'None'
        f_writer.writerow(res)
        
    for key, val in death_eaters_data.items():
        row = val
        res = [
            str(key), 
            row['name'], 
            row['blood_status'], 
            row['hair_color'], 
            row['eye_color'], 
            row['patronus'], 
            row['wand'],
            row['occupation'],
            row['loyalty'],
            row['house']
        ]
        for j in range(len(res)):
            if len(res[j]) == 0:
                res[j] = 'None'
        f_writer.writerow(res)        

In [59]:
name_lst = [row['name'] for row in has_relation_data_] + [row['name'] for row in death_eaters_data_]

In [60]:
with open('relations.csv', 'w', newline='') as f:
    f_writer = csv.writer(f)
    f_writer.writerow(['src', 'relation', 'dst'])
    
    for i in range(len(has_relation_data)):
        row = has_relation_data[i]
        src = row['name']
        
        if type(row['relation+']) == str:
            r_split = row['relation+'].split('(')
            if len(r_split) > 1:
                dst = r_split[0].strip()
                if dst in name_lst:
                    dst_idx = name_lst.index(dst)
                else:
                    continue
                curr_r = r_split[-1].split(')')[0]
                f_writer.writerow([i, curr_r, dst_idx])
        else:
            for r in row['relation+']:
                r_split = r.split('(')
            
                dst = r_split[0].strip()
                curr_r = r_split[-1].split(')')[0]
                
                if dst in name_lst:
                    dst_idx = name_lst.index(dst)
                else:
                    continue
        
                f_writer.writerow([i, curr_r, dst_idx])
    
        for dst in row['relation-']:
            if dst in name_lst:
                dst_idx = name_lst.index(dst)
            else:
                continue
            f_writer.writerow([i, 'enemy', dst_idx])
            
    for i, row in death_eaters_data.items():
        src = row['name']
        
        if type(row['relation+']) == str:
            r_split = row['relation+'].split('(')
            if len(r_split) > 1:
                dst = r_split[0].strip()
                if dst in name_lst:
                    dst_idx = name_lst.index(dst)
                else:
                    continue
                curr_r = r_split[-1].split(')')[0]
                f_writer.writerow([i, curr_r, dst_idx])
        else:
            for r in row['relation+']:
                r_split = r.split('(')
            
                dst = r_split[0].strip()
                curr_r = r_split[-1].split(')')[0]
                
                if dst in name_lst:
                    dst_idx = name_lst.index(dst)
                else:
                    continue
        
                f_writer.writerow([i, curr_r, dst_idx])
    
        for dst in row['relation-']:
            if dst in name_lst:
                dst_idx = name_lst.index(dst)
            else:
                continue
            f_writer.writerow([i, 'enemy', dst_idx])

In [230]:
with open('./github_structured_data/relations.csv') as f:
    github_relations = [row[:-1].split(',') for row in f][1:]
with open('./github_structured_data/characters.csv') as f:
    github_char_idx = [row[:-1].split(',')[1] for row in f][1:]

In [247]:
for row in data_:
    relation = row['relation+']
    
    if type(relation) == str: # 里面有一些是 str 
        if len(relation.split('(')) > 1:
            row['relation+'] = [relation.split('(')[0].strip()]
        else: # 有一个人的 relation 就是一个 mother
            row['relation+'] = []
    else:
        row['relation+'] = [i.split('(')[0].strip() for i in relation]

In [254]:
name_converter = {}
for row in data_:
    if row['name'] == 'Ronald Bilius Weasley':
        name_converter['Ron Weasley'] = 'Ronald Bilius Weasley'
        continue
        
    name = row['name'].split(' ')
    if len(name) > 2:
        name_converter[name[0] + ' ' +name[2]] = row['name']

In [256]:
for r in github_relations:
    src = r[0]
    dst = r[1]
    curr_r = r[2]
    
    srcLabel = github_char_idx[int(src)]
    dstLabel = github_char_idx[int(dst)]
    
    try:
        for row in data_:
            if srcLabel in name_converter.keys():
                if name_converter[srcLabel] == row['name']:
                    relation_type = 'relation' + curr_r
                
                    try:
                        if name_converter[dstLabel] not in row[relation_type]:
                            row[relation_type].append(name_converter[dstLabel])
                    except KeyError:
                        if dstLabel not in row[relation_type]:
                            row[relation_type].append(dstLabel)
    except:
        pass

现在fandom里都是4个houses的人 但基本上所有反派都不在 要不把wikidata的加进来？要不然很多relationship的destination都没有node

### data generation
- 用wikidata query 补全enemy那一栏
- 用enumerate(data_)生成index那一栏
- 看有没有entity是完全没有relation的 如果有就去除 因为生成的图也会是独立的一个node 所以没有意义
- 可以再参考一下github和kaggle的data 看看有什么要加的或者可以用的
- 生成rdf

### neo4j
- 用生成的data做图
- 图出来看看可不可以搞个search 可以看到specific 人物的所有特征和relation

### psl
- 想想怎么写rules
- attribites embedding
    - relation可以写成 p1,p2,+ 或者 p1,p2,-
    - loyalty 也可以写成这样
    - blood 可以写成 pureblood,1  muggle,0  half,0.5
    - etc...


### psl
hair_color, eye_color, skin_color 这些写成RGB?